In [1]:
from math import radians, sin, sqrt

import cadquery as cq
import ipywidgets as widgets
import jupyter_cadquery as jcq
from jupyter_cadquery.replay import replay, show

Overwriting auto display for cadquery Workplane and Shape


In [2]:
jcq.set_defaults(
    control="orbit",
    axes=True,
    axes0=True,
    grid=(True, False, False),
    glass=True,
    theme="browser",
)

# open_viewer("Cadquery")
replay_box = widgets.Checkbox(False, description="Enable Replay")
out = widgets.Output()
display(replay_box)
display(out)


@out.capture()
def changed(e):
    if e["new"]:
        jcq.replay.enable_replay(warning=False)
    else:
        jcq.replay.disable_replay()


replay_box.observe(changed, names="value")

Checkbox(value=False, description='Enable Replay')

Output()

In [3]:
# 45, 22
tolerance = 0.5
extrusions_w = 20
tape_w = 10
w = extrusions_w + tape_w
min_thickness = 1.0
safe_distance = 2.0
screw_d = 5
screw_head_d = 9 + tolerance

In [4]:
def make_mount():
    x = extrusions_w / 2 - screw_d / 2
    mount = (
        cq.Workplane("XY")
        .box(extrusions_w, extrusions_w, min_thickness + safe_distance)
        .faces(">Z")
        .workplane()
        .hole(screw_d)
        .pushPoints([(-x, 0), (x, 0)])
        .rect(screw_d, screw_d)
        .extrude(3)
    )
    return mount


replay(make_mount())

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'


CadViewerWidget(anchor=None, cad_width=640, glass=True, height=600, pinning=False, theme='browser', title=None…

In [5]:
def make_tape(l, w):
    r = (
        cq.Workplane("XY")
        .box(l, w, min_thickness)
        .vertices("<X and <Y and >Z")
        .workplane(centerOption="CenterOfMass")
        .rect(l, 0.5, centered=False)
        .extrude(safe_distance)
    )
    return r


replay(make_tape(50, 10))

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'


CadViewerWidget(anchor=None, cad_width=640, glass=True, height=600, pinning=False, theme='browser', title=None…

In [6]:
r = (
    cq.Assembly()
    .add(make_mount(), name="mount")
    .add(make_tape(50, 10), name="tape", color=cq.Color("green"))
)
r.constrain("mount@faces@|Z", "tape@faces@|Z", "Axis", param=0)
r.constrain("mount@faces@|Y", "tape@faces@|Y", "Axis", param=0)
r.constrain("mount@edges@>Y and <Z", "tape@edges@<Z and <Y", "Point")
r.solve()
show(r)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       18

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality co

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…

In [25]:
def make_t_connector_cover(base_only=False):
    l1 = 60 + tolerance * 2
    l2 = extrusions_w - 1 * 2
    l3 = extrusions_w - 1
    t1 = 2
    t2 = 3
    ext = 7  # .5
    hole_pos = l1 / 2 + extrusions_w / 2 + ext / 2
    half_points = [
        (l2 / 2, min_thickness + t1 + t2),
        (l1 / 2 + extrusions_w + ext - 0.0001, min_thickness + safe_distance),
        (l1 / 2 + extrusions_w + ext, min_thickness + safe_distance),
    ]
    curve = list(reversed(half_points)) + list(
        map(lambda x: (x[0] * -1, x[1]), half_points)
    )
    side_wire = (
        cq.Workplane("XZ")
        .spline(curve)
        .line(0, -min_thickness - safe_distance)
        .lineTo(l1 / 2 + extrusions_w + ext, 0)
        .close()
    )
    r = (
        side_wire.extrude(extrusions_w + tape_w)
        .faces("<Z")
        .vertices("<X and<Y")
        .workplane(centerOption="CenterOfMass", invert=True)
        .rect(l1 + extrusions_w * 2 + ext * 2, tape_w, centered=False)
        .cutBlind(safe_distance)
        .faces("<Z")
        .workplane(centerOption="CenterOfMass", offset=-(min_thickness + safe_distance))
        .pushPoints([(-hole_pos, 0), (hole_pos, 0)])
        .hole(screw_head_d)
        .faces("<Z")
        .workplane()
        .pushPoints([(-hole_pos, 0), (hole_pos, 0)])
        .hole(screw_d)
    )
    if not base_only:
        r = (
            r.faces("<Z")
            .edges(">Y")
            .workplane(centerOption="CenterOfMass")
            .rect(l2, extrusions_w + tape_w, centered=(True, False))
            .cutBlind(-t1)
            .workplane()
            .rect(l1, l3, centered=(True, False))
            .cutThruAll()
            .faces(">Z[1]")
            .edges("<Y")
            .workplane(centerOption="CenterOfMass", invert=True)
            .rect(tape_w, tape_w, centered=(True, False))
            .cutBlind(t2)
        )
    return r


replay(make_t_connector_cover())

Use the multi select box below to select one or more steps you want to examine


In [26]:
ptfe_tube_mount = make_t_connector_cover(base_only=True).faces(">Y").edges("<Z").workplane(centerOption="CenterOfMass").center(0, 4/2+1).hole(4)

replay(ptfe_tube_mount)

Use the multi select box below to select one or more steps you want to examine


In [9]:
def make_base(l):
    r = (
        cq.Workplane("XY")
        .rect(w, l, centered=False)
        .extrude(min_thickness)
        .faces(">Z")
        .vertices("<X and <Y")
        .rect(extrusions_w, l, centered=False)
        .extrude(safe_distance)
    )
    return r


replay(make_base(50))

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'


CadViewerWidget(anchor=None, cad_width=640, glass=True, height=600, pinning=False, theme='browser', title=None…

In [10]:
def normal(l=50, holes=1):
    if holes == 1:
        return (
            make_base(l)
            .faces(">Z")
            .workplane(centerOption="CenterOfBoundBox")
            .hole(screw_d)
            .faces(">Z")
            .workplane(centerOption="CenterOfBoundBox")
            .pushPoints([(0, l / 2 - screw_d / 2 - 1), (0, -l / 2 + screw_d / 2 + 1)])
            .circle(screw_d / 2)
            .extrude(3)
        )
    else:
        pos = [(l - extrusions_w) / (holes - 1) * i for i in range(holes)]
        pos = [(0, n - l / 2 + extrusions_w / 2) for n in pos]
        print(pos)
        return (
            make_base(l)
            .faces(">Z")
            .workplane(centerOption="CenterOfBoundBox")
            .pushPoints(pos)
            .hole(screw_d)
        )


replay(normal())

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'


CadViewerWidget(anchor=None, cad_width=640, glass=True, height=600, pinning=False, theme='browser', title=None…

In [11]:
replay(normal(l=100, holes=3))

[(0, -40.0), (0, 0.0), (0, 40.0)]
Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'


CadViewerWidget(anchor=None, cad_width=640, glass=True, height=600, pinning=False, theme='browser', title=None…

In [12]:
def corner(l1=60 + 2 + tolerance, l2=60 + 2 + tolerance, ext=extrusions_w):
    l_a = l1 + ext
    l_b = l2 + ext
    a = (
        make_base(l_a)
        .faces(">Z")
        .workplane(centerOption="CenterOfBoundBox")
        .pushPoints([(0, l_a / 2 - ext / 2)])
        .hole(screw_d)
    )
    b = (
        make_base(l_b)
        .faces(">Z")
        .workplane(centerOption="CenterOfBoundBox")
        .pushPoints([(0, -(l_b / 2 - ext / 2))])
        .hole(screw_d)
        .rotate((0, 0, 0), (0, 0, 1), 90)
        .translate((l_b, 0))
    )
    r = (
        cq.Workplane("XY")
        .add(a)
        .add(b)
        .faces(">Z")
        .workplane()
        .rect(extrusions_w - 1, l1, centered=False)
        .cutBlind(-99)
        .workplane()
        .rect(l2, extrusions_w - 1, centered=False)
        .cutThruAll()
    )
    return r


replay(corner())

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'


CadViewerWidget(anchor=None, cad_width=640, glass=True, height=600, pinning=False, theme='browser', title=None…

In [27]:
cq.exporters.export(make_t_connector_cover(), "/tmp/a.stl")
cq.exporters.export(corner(), "/tmp/b.stl")
cq.exporters.export(normal(345, holes=5), "/tmp/c.stl")
cq.exporters.export(normal(142, holes=3), "/tmp/c142.stl")
cq.exporters.export(normal(115, holes=2), "/tmp/c115.stl")
cq.exporters.export(normal(26.5, holes=1), "/tmp/d.stl")
cq.exporters.export(ptfe_tube_mount, "/tmp/e.stl")

[(0, -162.5), (0, -81.25), (0, 0.0), (0, 81.25), (0, 162.5)]
[(0, -61.0), (0, 0.0), (0, 61.0)]
[(0, -47.5), (0, 47.5)]
